# STEP 1 : Set up the four study cases

# STEP 2 : Load and align all input data

In [ ]:
!pip -q install pyomo amplpy
!python -m amplpy.modules install coin -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.8 MB/s eta 0:00:00


In [ ]:
from amplpy import modules
import pandas as pd
import pyomo.environ as pyo

In [ ]:
# Read CSV files
demand_df = pd.read_csv("Summer_Berlin_Aggregated_Demand.csv", sep=';', decimal=',')

In [ ]:
pv_df   = pd.read_csv("Berlin_Summer_Week_01.07_07.07_2023.csv")

pv_ref_kwp = 1000.0  # your PV file corresponds to a ~1000 kWp reference plant

pv_df["pv_kw"] = pv_df["Power"].astype(float) / 1000.0          # W -> kW (for the reference plant)
pv_df["pv_per_kwp"] = pv_df["pv_kw"] / pv_ref_kwp               # kW per 1 kWp installed



In [ ]:
price_df  = pd.read_csv("Hourly_Market_Prize_Berlin_Summer_Week.csv")


In [ ]:
#This creates an index for looping over time steps for the three files:
T = len(demand_df)
assert len(pv_df) == T, "PV file length != demand length"
assert len(price_df) == T, "Price file length != demand length"
t_idx = range(T)

# turning the columns in our datatables  into numpay array vectors so we can use them easily in an optimization model.
D  = demand_df["aggregate_demand"].to_numpy()

PV = pv_df["pv_per_kwp"].to_numpy(dtype=float)                  # <-- THIS is what the model should use
P  = price_df["price_eur_per_kwh"].to_numpy()

In [ ]:
print("PV per kWp max:", PV.max())

PV per kWp max: 0.7637999999999999


In [ ]:
# we assign the Economic parameters (same for all cases)
econ = {
    # discount rate
    "discount_rate": 0.05,   # 5%
    "weeks_per_year": 52,

    # PV (investment cost)
    "pv_capex_eur_per_kwp": 900,       # €/kWp
    "pv_opex_frac_per_year": 0.015,    # 1.5% of CAPEX per year
    "pv_lifetime_years": 25,

    # Battery / ESS (investment cost)
    "bat_capex_eur_per_kwh": 450,      # €/kWh
    "bat_opex_frac_per_year": 0.02,    # 2% of CAPEX per year
    "bat_lifetime_years": 15}

In [ ]:
# CRF is calculated to convert the upfront investment into yearly annual payment
def crf(r: float, n_years: int) -> float:
  return (r * (1 + r) ** n_years) / ((1 + r) ** n_years - 1)
# convert the annual payment into weekly payment
def weeklyized_cost(capex: float, opex_frac_per_year: float, r: float, n_years: int, weeks_per_year: int = 52) -> float:
    annualised = capex * crf(r, n_years) + capex * opex_frac_per_year
    return annualised / weeks_per_year

In [ ]:
# We assign the parameters for the battery
eta = 0.90                      # energy transfer efficiency
soc_min_frac = 0.10             # 10%
soc_max_frac = 0.90             # 90%
soc_init_frac = 0.50            # 50%
soc_final_frac = 0.50           # 50%
c_rate = 1.0                 # the total stored energy that the  battery can charge or discharge  in 1 hour
#  a penalty factor imposed on consumers for selling the electricity to the grid.
sell_factor = 0.50
# we are creating a new vector where each
#hourly selling price equals 0.5 × the corresponding hourly buying/market price in P (element-by-element).
P_sell = sell_factor * P

# STEP 3  Decision variables + Constraints

In [ ]:
m = pyo.ConcreteModel()

In [ ]:
# defining the sets
m.T = pyo.RangeSet(0, T-1)
m.Tsoc = pyo.RangeSet(0, T)  # defining another set for the state of charge

In [ ]:
# defining the parameters
m.D = pyo.Param(m.T, initialize={i: float(D[i]) for i in t_idx})
m.PV_unit = pyo.Param(m.T, initialize={i: float(PV[i]) for i in t_idx})
m.p_buy = pyo.Param(m.T, initialize={i: float(P[i]) for i in t_idx})
# defining the selling price parameter
m.p_sell = pyo.Param(m.T, initialize={i: float(P_sell[i]) for i in t_idx})

In [ ]:
#Decision variables
# PV and battery capacity
m.PV_cap = pyo.Var(domain=pyo.NonNegativeReals)
m.E_cap  = pyo.Var(domain=pyo.NonNegativeReals)

# PV to Load , PV to Battery and, PV to Grid
m.pv2load = pyo.Var(m.T, domain=pyo.NonNegativeReals)
m.pv2batt = pyo.Var(m.T, domain=pyo.NonNegativeReals)
m.pv_exp  = pyo.Var(m.T, domain=pyo.NonNegativeReals)

# grid to load and grid battery
m.g2load  = pyo.Var(m.T, domain=pyo.NonNegativeReals)
m.g2batt  = pyo.Var(m.T, domain=pyo.NonNegativeReals)

#total charge, state of discharge ,and state of charge.
m.ch  = pyo.Var(m.T, domain=pyo.NonNegativeReals)
m.dis = pyo.Var(m.T, domain=pyo.NonNegativeReals)
m.soc = pyo.Var(m.Tsoc, domain=pyo.NonNegativeReals)

# binary variable used to avoid the simultanious charge and discharge
m.u = pyo.Var(m.T, domain=pyo.Binary)

In [ ]:
#Constraint
#Linking the PV profile
def pv_split_rule(m,i):
  return m.pv2load[i] + m.pv2batt[i] + m.pv_exp[i] == m.PV_unit[i] * m.PV_cap
m.pv_split = pyo.Constraint(m.T, rule=pv_split_rule)

# linking the demand
def demand_balance_rule(m, i):
    return m.pv2load[i] + m.dis[i] + m.g2load[i] == m.D[i]
m.demand_balance = pyo.Constraint(m.T, rule=demand_balance_rule)

#linking the battery
def charge_def_rule(m, i):
    return m.ch[i] == m.pv2batt[i] + m.g2batt[i]
m.charge_def = pyo.Constraint(m.T, rule=charge_def_rule)

#updating the state of charge
def soc_update_rule(m, i):
    return m.soc[i+1] == m.soc[i] + eta * m.ch[i] - (1/eta) * m.dis[i]
m.soc_update = pyo.Constraint(m.T, rule=soc_update_rule)

#defining the limits of charging and discharging of the battery
def soc_min_rule(m, k):
    return m.soc[k] >= soc_min_frac * m.E_cap
def soc_max_rule(m, k):
    return m.soc[k] <= soc_max_frac * m.E_cap
m.soc_min = pyo.Constraint(m.Tsoc, rule=soc_min_rule)
m.soc_max = pyo.Constraint(m.Tsoc, rule=soc_max_rule)

#Initial and final SOC
m.soc_init = pyo.Constraint(expr=m.soc[0] == soc_init_frac * m.E_cap)
m.soc_final = pyo.Constraint(expr=m.soc[T] == soc_final_frac * m.E_cap)

#assigning constraints to the charge and discharge , and enforcing the non simultanous charging or discharging
m.P_cap = pyo.Var(domain=pyo.NonNegativeReals)
m.pcap_link = pyo.Constraint(expr=m.P_cap == c_rate * m.E_cap)
D_avg = float(D.mean())      # kWh/h ≈ kW
E_ub  = 12.0 * D_avg         # kWh (≈ 12 hours of average load)

m.E_cap.setub(E_ub)
print("E_ub =", E_ub)
M = float(c_rate * E_ub)   # constant big-M from your E_cap upper bound

m.ch_size  = pyo.Constraint(m.T, rule=lambda m,t: m.ch[t]  <= m.P_cap)
m.dis_size = pyo.Constraint(m.T, rule=lambda m,t: m.dis[t] <= m.P_cap)

m.ch_onoff  = pyo.Constraint(m.T, rule=lambda m,t: m.ch[t]  <= m.u[t] * M)
m.dis_onoff = pyo.Constraint(m.T, rule=lambda m,t: m.dis[t] <= (1-m.u[t]) * M)



E_ub = 6700.110000000001


In [ ]:
# --- Export limit from demand scale ---
P_export_max = float(D.max())    # kW equivalent (kWh/h)

# Limit hourly exported energy (kWh) because Δt=1h
m.export_limit = pyo.Constraint(m.T, rule=lambda m, t: m.pv_exp[t] <= P_export_max)

In [ ]:
# grid fee for using the network
grid_fee_eur_per_kwh = 0.0

In [ ]:
# the economic parameters + its values
pv_weekly_cost_per_kwp = weeklyized_cost(
    capex=econ["pv_capex_eur_per_kwp"],
    opex_frac_per_year=econ["pv_opex_frac_per_year"],
    r=econ["discount_rate"],
    n_years=econ["pv_lifetime_years"],
    weeks_per_year=econ["weeks_per_year"]
)

bat_weekly_cost_per_kwh = weeklyized_cost(
    capex=econ["bat_capex_eur_per_kwh"],
    opex_frac_per_year=econ["bat_opex_frac_per_year"],
    r=econ["discount_rate"],
    n_years=econ["bat_lifetime_years"],
    weeks_per_year=econ["weeks_per_year"]
)


In [ ]:
# Expressions for the objective function
def grid_cost_expr(m):
    return sum((m.p_buy[i] + grid_fee_eur_per_kwh) * (m.g2load[i] + m.g2batt[i]) for i in m.T)

def export_revenue_expr(m):
    # revenue from PV exported at penalty selling price
    return sum(m.p_sell[i] * m.pv_exp[i] for i in m.T)

def weekly_capex_expr(m):
    # weekly equivalent of annualised PV+ESS investment
    return pv_weekly_cost_per_kwp * m.PV_cap + bat_weekly_cost_per_kwh * m.E_cap

m.C_grid = pyo.Expression(rule=grid_cost_expr)
m.R_exp  = pyo.Expression(rule=export_revenue_expr)
m.C_capex = pyo.Expression(rule=weekly_capex_expr)

In [ ]:
# Total weekly cost = grid cost - export revenue + weekly capex
m.total_cost = pyo.Expression(expr=m.C_grid - m.R_exp + m.C_capex)

#The objective function
m.obj = pyo.Objective(expr=m.total_cost, sense=pyo.minimize)

In [ ]:
#Use the glpk solver
!apt-get -qq update
!apt-get -qq install -y glpk-utils
import pyomo.environ as pyo
solver = pyo.SolverFactory("glpk", executable="/usr/bin/glpsol")  # Explicitly set the path to glpsol
# Removed solver.options["seconds"] as it was specific to CBC and might not apply to GLPK
res = solver.solve(m)
print("PV_cap =", pyo.value(m.PV_cap))
print("E_cap  =", pyo.value(m.E_cap))
print("Weekly total cost =", pyo.value(m.total_cost))
print("  Grid cost =", pyo.value(m.C_grid))
print("  Export revenue =", pyo.value(m.R_exp))
print("  Weekly capex =", pyo.value(m.C_capex))

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
PV_cap = 2429.59124087591
E_cap  = 177.769887208029
Weekly total cost = 7081.752562639731
  Grid cost = 4110.904609259396
  Export revenue = 822.4861534079491
  Weekly capex = 3793.3341067882834
